# load required modules

In [1]:
import sys, os, pygmt
sys.path.insert(0, '/home/581/da1339/AFIM/src/AFIM/src')
from fast_ice_processor import FastIceProcessor
from pack_ice_processor import PackIceProcessor
from sea_ice_plotter    import SeaIcePlotter
import numpy            as np
import pandas           as pd
import xarray           as xr
from datetime           import timedelta, date, datetime

# process for fast/pack ice

In [2]:
dt0_str  = "1993-01-01"
dtN_str  = "1993-01-31"
sim_name = 'ktens-max'
mask_fi  = True
FI_proc  = FastIceProcessor(sim_name = sim_name,
                            dt0_str  = dt0_str,
                            dtN_str  = dtN_str)
FI       = FI_proc.process_window(ow_zarrs=True)

2025-04-03 07:32:24,026 - INFO - loading data for date period: 1993-01-01 00:00:00 to 1993-01-15 00:00:00
INFO:ktens-max:loading data for date period: 1993-01-01 00:00:00 to 1993-01-15 00:00:00
2025-04-03 07:32:24,031 - INFO - loading CSV climatology
INFO:ktens-max:loading CSV climatology
2025-04-03 07:32:24,041 - INFO - using gridded climatology
INFO:ktens-max:using gridded climatology
2025-04-03 07:32:24,042 - INFO - Averaged observational gridded climatology already exists
/g/data/gv90/da1339/SeaIce/FI_obs/AF_FI_2020db_18yrAvg_gridded.zarr
INFO:ktens-max:Averaged observational gridded climatology already exists
/g/data/gv90/da1339/SeaIce/FI_obs/AF_FI_2020db_18yrAvg_gridded.zarr
2025-04-03 07:32:24,246 - INFO - selecting 1-DOY from gridded climatology to associate with this model period
INFO:ktens-max:selecting 1-DOY from gridded climatology to associate with this model period
2025-04-03 07:32:24,250 - INFO - constructing list of CICE files to load for this period
INFO:ktens-max:cons

1D vars looks like: {'FIA': <xarray.DataArray 'truediv-a7b66dba0b508a56cea7201564ca7df3' (t_fi: 1)> Size: 8B
dask.array<truediv, shape=(1,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * t_fi     (t_fi) datetime64[ns] 8B 1993-01-01
Attributes:
    units:          1000-km^2
    long_name:      fast ice area
    description:    sea ice area summed over spatial extent and masked with f...
    CICE_variable:  aice
    dimensions:     1D, 'FIE': <xarray.DataArray 'truediv-d8d833b38e27341ff65e3c5f3d939cf3' (t_fi: 1)> Size: 8B
dask.array<truediv, shape=(1,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * t_fi     (t_fi) datetime64[ns] 8B 1993-01-01
Attributes:
    units:          1000-km^2
    long_name:      fast ice extent
    description:    sea ice extent summed over spatial extent and masked with...
    CICE_variable:  aice
    dimensions:     1D, 'FIV': <xarray.DataArray 'sum-aggregate-6d0dff28278817aa7ca419bc342c5ba4' (t_fi: 1)> Size

MergeError: conflicting values for variable 'lat' on objects to be combined. You can skip this check by specifying compat='override'.

# plot 1D analysis

In [ ]:
dt0_str  = "1993-01-01"
dtN_str  = "1993-12-31"
sim_name = 'ktens-max'
plotter  = SeaIcePlotter(sim_name, dt0_str, dtN_str, show_figs=True, ice_type='PI', mask_fast_ice=True)
plotter.plot_timeseries()

In [ ]:
#plotter.plot_timeseries(var_names=["PI_STRESS"])
plotter.plot_timeseries_compare(
    comparison_name = "tensile-stars",
    sim_names       = ["baseline", "ktens-max", "Pstar-max", "Cstar-min"],
    var_name        = "PIA",
    label_dict      = {"baseline"  : "tensile = 0.2, Pstar = 2.75e4, Cstar = 20",
                       "ktens-max" : "tensile = 0.3, Pstar = 2.75e4, Cstar = 20",
                       "Pstar-max" : "tensile = 0.2, Pstar = 5e4   , Cstar = 20",
                       "Cstar-min" : "tensile = 0.2, Pstar = 2.75e4, Cstar = 10"})

# plot 2D analysis (requires existing FI/PI dataset)

In [ ]:
dt0_str  = "1993-01-01"
dtN_str  = "1999-12-31"
sim_names = ["baseline", "ustar-low", "Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min",
             "ktens-nil", "ktens-min", "ktens-max", "gi-max", "gi-min",
             "Rothrock", "elps-pack", "elps-fast", "elps-max", "elps-def", "elps-mid" ]
for sim_name in sim_names:
    SI_plot = SeaIcePlotter(sim_name, dt0_str, dtN_str, show_figs=False, overwrite=True, single_figure=False)
    SI_plot.prepare_data_for_plotting(var_name='FIP')
    SI_plot.plot_regions()

# generate PBS jobs for analysis

In [ ]:
from pathlib import Path

sim_names = ["Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min", "Rothrock", "baseline",
             "elps-def", "elps-fast", "elps-max", "elps-mid", "elps-pack", "gi-max",
             "gi-min", "ktens-max", "ktens-min", "ktens-nil", "oras-sfc-ctl", "ustar-low", "van_achter"]
pbs_template = """#!/bin/bash
#PBS -N PI-proc-{name}
#PBS -P gv90
#PBS -l walltime=24:00:00
#PBS -q normalbw
#PBS -l mem=32GB
#PBS -l ncpus=28
#PBS -l storage=gdata/cj50+gdata/jk72+scratch/jk72+gdata/ik11+gdata/hh5+gdata/rt52+gdata/gb6+gdata/gv90
#PBS -M daniel.atwater@utas.edu.au

module purge
module use /g/data/hh5/public/modules
module load conda/analysis3

cd ~/AFIM/src/AFIM/scripts
python3 ./compute_pack_ice.py {name}
"""

output_dir = Path("/home/581/da1339/AFIM/src/sh/analysis")
output_dir.mkdir(exist_ok=True)

for sim in sim_names:
    script_content = pbs_template.format(name=sim)
    script_path = Path(output_dir,f"PI-proc_{sim}.pbs")
    with open(script_path, "w") as f:
        f.write(script_content)
print(f"✓ Generated {len(sim_names)} PBS scripts in: {output_dir.resolve()}")